<a href="https://colab.research.google.com/github/jarrett707/lyricslab/blob/main/LyricsLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/PratikSaha198/Lyrics-Generation-using-RNNs

Cloning into 'Lyrics-Generation-using-RNNs'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 58 (delta 27), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (58/58), 134.68 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [2]:
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
path_to_file = 'lyrics_dataset.txt'

In [6]:
# Opening the text file in read mode and standard encoding it
text = open('spotify_millsongdata.csv', 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 3145728 characters


In [7]:
# A look at the first 250 characters in text
print(text[:100])

artist,song,link,text
ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look a


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

80 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
print('{ ===========>')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n==========>}')

{ ===========>
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  "'" :   5,
  '(' :   6,
  ')' :   7,
  '+' :   8,
  ',' :   9,
  '-' :  10,
  '.' :  11,
  '/' :  12,
  '0' :  13,
  '1' :  14,
  '2' :  15,
  '3' :  16,
  '4' :  17,
  '5' :  18,
  '6' :  19,
  ...
==========>}


In [11]:
# Show how the first 20 characters from the text are mapped to integers
print ('{} ==> characters mapped to int ==> {}'.format(repr(text[:20]), text_as_int[:20]))

'artist,song,link,tex' ==> characters mapped to int ==> [54 71 73 62 72 73  9 72 68 67 60  9 65 62 67 64  9 73 58 77]


In [12]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()] , end = "")

artis

In [13]:
# Using batch method converted individual characters to sequences of desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'artist,song,link,text\r\nABBA,Ahe\'s My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at'
" her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she "
"smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes "
"me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without"
" her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go fo"


In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'artist,song,link,text\r\nABBA,Ahe\'s My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look a'
Target data: 'rtist,song,link,text\r\nABBA,Ahe\'s My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at'


In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1500 # keep between (1024 -> 1800) for best results

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([

    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),

    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),

    tf.keras.layers.Dense(vocab_size,activation='relu'),

    tf.keras.layers.Dropout(0.2),
  ])
  return model

In [19]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 80) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           20480     
                                                                 
 gru (GRU)                   (64, None, 1500)          7911000   
                                                                 
 dense (Dense)               (64, None, 80)            120080    
                                                                 
 dropout (Dropout)           (64, None, 80)            0         
                                                                 
Total params: 8051560 (30.71 MB)
Trainable params: 8051560 (30.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([22, 64, 24, 17, 45, 50, 75, 32, 72, 11, 57, 40, 17, 79,  2, 64, 39,
       55, 23, 76, 53, 72,  7, 19, 75,  8, 56, 33, 48, 53, 17, 11, 42, 26,
       76, 11, 69, 36,  3, 60, 46, 28, 20, 50, 48, 53, 74, 15, 71, 29, 27,
       64, 47, 36,  6, 16, 56, 24, 47, 73, 16, 69, 51, 41, 16, 57, 15, 15,
       19, 56, 54, 69, 49, 73, 53,  2, 71, 36, 41, 41, 57, 25, 41,  1,  8,
       19, 67, 49, 43, 29, 29, 60, 67, 31, 42,  5, 26, 79, 26, 44])

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'st a state of mind  \r\nAnd who knows what it is that we might find  \r\nIf we try  \r\nAnd who knows what'

Next Char Predictions: 
 "9k?4UZvHs.dP4z kOb:w_s)6v+cIX_4.RBw.pL!gVD7ZX_u2rECkWL(3c?Wt3p[Q3d226capYt_ rLQQdAQ\r+6nYSEEgnGR'BzBT"


In [24]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 80)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3823156


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            20480     
                                                                 
 gru_1 (GRU)                 (1, None, 1500)           7911000   
                                                                 
 dense_1 (Dense)             (1, None, 80)             120080    
                                                                 
 dropout_1 (Dropout)         (1, None, 80)             0         
                                                                 
Total params: 8051560 (30.71 MB)
Trainable params: 8051560 (30.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:

def generate_text(model, chars_to_generate , temp , start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = chars_to_generate

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = temp

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [33]:
from numpy import arange

# Number of characters to generate (keep between 250 to 500)
chars_to_generate = 500

# Printing the generated text
# Temperature 1.0 gives the craziest output and 0.1 gives the lowest varience
# Keeping the temperature 0.35 gives best meaningful / coherent text.

# Give the seed string as the first word of generate text
print(generate_text(model , chars_to_generate , 0.35 , start_string=u"Hate "))

# Uncomment below to check the variences ==>

# for i in arange(0.1,1.1,0.1):
#   print("==============")
#   print("FOR TEMP : {} ".format(i))
#   print("==============")
#   print(generate_text(model , chars_to_generate , i , start_string=u"Love "))
#   print()


Hate and me  
And a real your brade  
And you be a love  
And when you want to live  
1way you are in you  
  
And when you're not can!  
  
And chance was to be the start  
  
And the way you love me  
  
And the eas  
And then you want to be a broke  
  
And it's a man  
And when you to be back to me  
And when you be a shere  
And when you're not the beat  
  
And she's a start and the say  
And the sing and show the start  
  
So you be the say  
And you're hard to say  

